In [87]:
try:
  import mlagents
  print("ml-agents already installed")
except ImportError:
  !python -m pip install -q mlagents==
  print("Installed ml-agents")

ml-agents already installed


In [88]:
env_id = "C:/Users/User/Desktop/MLagents_18/Test2/3Dball"

In [89]:
try:
  env.close()
except:
  pass
# -----------------

In [90]:
from mlagents_envs.environment import UnityEnvironment

In [91]:
env = UnityEnvironment(file_name=env_id, seed=1, side_channels=[])

In [92]:
env.reset()
env.reset()
behavior_name = list(env.behavior_specs)[0]
print(f"Name of the behavior : {behavior_name}")
spec = env.behavior_specs[behavior_name]

Name of the behavior : Basic?team=0


In [93]:
print("Number of observations : ", len(spec.observation_specs))

# Is there a visual observation ?
# Visual observation have 3 dimensions: Height, Width and number of channels
vis_obs = any(len(spec.shape) == 3 for spec in spec.observation_specs)
print("Is there a visual observation ?", vis_obs)

Number of observations :  1
Is there a visual observation ? False


In [94]:
# Is the Action continuous or multi-discrete ?
if spec.action_spec.continuous_size > 0:
  print(f"There are {spec.action_spec.continuous_size} continuous actions")
if spec.action_spec.is_discrete():
  print(f"There are {spec.action_spec.discrete_size} discrete actions")


# How many actions are possible ?
#print(f"There are {spec.action_size} action(s)")

# For discrete actions only : How many different options does each action has ?
if spec.action_spec.discrete_size > 0:
  for action, branch_size in enumerate(spec.action_spec.discrete_branches):
    print(f"Action number {action} has {branch_size} different options")


There are 1 discrete actions
Action number 0 has 3 different options


In [95]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [96]:
env.set_actions(behavior_name, spec.action_spec.empty_action(len(decision_steps)))

In [97]:
env.step()

In [98]:
!pip install matplotlib

In [99]:
import matplotlib.pyplot as plt
%matplotlib inline

for index, obs_spec in enumerate(spec.observation_specs):
  if len(obs_spec.shape) == 3:
    print("Here is the first visual observation")
    plt.imshow(decision_steps.obs[index][0,:,:,:])
    plt.show()

for index, obs_spec in enumerate(spec.observation_specs):
  if len(obs_spec.shape) == 1:
    print("First vector observations : ", decision_steps.obs[index][0,:])

First vector observations :  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [100]:
for episode in range(10):
  env.reset()
  decision_steps, terminal_steps = env.get_steps(behavior_name)
  tracked_agent = -1 # -1 indicates not yet tracking
  done = False # For the tracked_agent
  episode_rewards = 0 # For the tracked_agent
  while not done:
    # Track the first agent we see if not tracking
    # Note : len(decision_steps) = [number of agents that requested a decision]
    if tracked_agent == -1 and len(decision_steps) >= 1:
      tracked_agent = decision_steps.agent_id[0]

    # Generate an action for all agents
    action = spec.action_spec.random_action(len(decision_steps))

    # Set the actions
    env.set_actions(behavior_name, action)

    # Move the simulation forward
    env.step()

    # Get the new simulation results
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    if tracked_agent in decision_steps: # The agent requested a decision
      episode_rewards += decision_steps[tracked_agent].reward
    if tracked_agent in terminal_steps: # The agent terminated its episode
      episode_rewards += terminal_steps[tracked_agent].reward
      done = True
  print(f"Total rewards for episode {episode} is {episode_rewards}")

Total rewards for episode 0 is 0.6800000164657831
Total rewards for episode 1 is 0.05000000447034836
Total rewards for episode 2 is 0.8800000119954348
Total rewards for episode 3 is 0.850000012665987
Total rewards for episode 4 is -0.1899999901652336
Total rewards for episode 5 is -0.16999999061226845
Total rewards for episode 6 is -0.3499999865889549
Total rewards for episode 7 is -0.019999993965029716
Total rewards for episode 8 is -0.38999998569488525
Total rewards for episode 9 is 0.1900000274181366


In [101]:
env.close()
print("Closed environment")

Closed environment
